# RNN 3D: x-collapsed

In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import numpy as np
import glob
import nibabel as nib
import os
import matplotlib.pyplot as plt
import scipy.ndimage
import random
from tensorflow.keras.layers import Dropout, Dense, Reshape, Flatten, Conv3D, Conv3DTranspose, LeakyReLU, Input, Embedding, multiply, Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization, Bidirectional, AdditiveAttention, LayerNormalization
from functools import partial
from tensorflow.keras import models, layers
import pandas as pd

In [2]:
full_schizophrenia_ids = [
    'A00009280', 'A00028806', 'A00023132', 'A00014804', 'A00016859', 'A00021598', 'A00001181', 'A00023158',
    'A00024568', 'A00028405', 'A00001251', 'A00000456', 'A00015648', 'A00002405', 'A00027391', 'A00016720',
    'A00018434', 'A00016197', 'A00027119', 'A00006754', 'A00009656', 'A00038441', 'A00012767', 'A00034273',
    'A00028404', 'A00035485', 'A00024684', 'A00018979', 'A00027537', 'A00004507', 'A00001452', 'A00023246',
    'A00027410', 'A00014719', 'A00024510', 'A00000368', 'A00019293', 'A00014830', 'A00015201', 'A00018403',
    'A00037854', 'A00024198', 'A00001243', 'A00014590', 'A00002337', 'A00024953', 'A00037224', 'A00027616',
    'A00001856', 'A00037619', 'A00024228', 'A00038624', 'A00037034', 'A00037649', 'A00022500', 'A00013216',
    'A00020787', 'A00028410', 'A00002480', 'A00028303', 'A00020602', 'A00024959', 'A00018598', 'A00014636',
    'A00019349', 'A00017147', 'A00023590', 'A00023750', 'A00031597', 'A00015518', 'A00018317', 'A00016723',
    'A00021591', 'A00023243', 'A00017943', 'A00023366', 'A00014607', 'A00020414', 'A00035003', 'A00028805',
    'A00029486', 'A00000541', 'A00028408', 'A00000909', 'A00031186', 'A00000838' ]

# schizohrenia_id that satisfy t>90, 59 in total
met_requirement_schizophrenia_ids = [
    'A00000368', 'A00000456', 'A00000541', 'A00000838', 'A00001251', 'A00001452', 'A00004507',
    'A00006754', 'A00009280', 'A00012767', 'A00013216', 'A00014607', 'A00014719', 'A00014804',
    'A00014830', 'A00015201', 'A00015648', 'A00016197', 'A00016720', 'A00016723', 'A00017147',
    'A00018317', 'A00018403', 'A00018434', 'A00018979', 'A00019293', 'A00020414', 'A00020602', 
    'A00020787', 'A00021591', 'A00021598', 'A00023158', 'A00023246', 'A00023590', 'A00023750', 
    'A00024198', 'A00024228', 'A00024568', 'A00024684', 'A00024953', 'A00024959', 'A00027410', 
    'A00027537', 'A00028303', 'A00028404', 'A00028408', 'A00028805', 'A00028806', 'A00031186', 
    'A00031597', 'A00034273', 'A00035003', 'A00035485', 'A00037034', 'A00037224', 'A00037619', 
    'A00037649', 'A00038441', 'A00038624']

full_control_ids = [
    'A00007409', 'A00013140', 'A00021145', 'A00036049', 'A00022810', 'A00002198', 'A00020895', 'A00004667',
    'A00015826', 'A00023120', 'A00022837', 'A00010684', 'A00009946', 'A00037318', 'A00033214', 'A00022490',
    'A00023848', 'A00029452', 'A00037564', 'A00036555', 'A00023095', 'A00022729', 'A00024955', 'A00024160',
    'A00011725', 'A00027487', 'A00024446', 'A00014898', 'A00015759', 'A00028409', 'A00017294', 'A00014522',
    'A00012995', 'A00031764', 'A00025969', 'A00033147', 'A00018553', 'A00023143', 'A00036916', 'A00028052',
    'A00023337', 'A00023730', 'A00020805', 'A00020984', 'A00000300', 'A00010150', 'A00024932', 'A00035537',
    'A00022509', 'A00028406', 'A00004087', 'A00035751', 'A00023800', 'A00027787', 'A00022687', 'A00023866',
    'A00021085', 'A00022619', 'A00036897', 'A00019888', 'A00021058', 'A00022835', 'A00037495', 'A00026945',
    'A00018716', 'A00026907', 'A00023330', 'A00016199', 'A00037238', 'A00023131', 'A00014120', 'A00021072',
    'A00037665', 'A00022400', 'A00003150', 'A00024372', 'A00021081', 'A00022592', 'A00022653', 'A00013816',
    'A00014839', 'A00031478', 'A00014225', 'A00013363', 'A00037007', 'A00020968', 'A00024301', 'A00024820',
    'A00035469', 'A00029226', 'A00022915', 'A00022773', 'A00024663', 'A00036844', 'A00009207', 'A00024535',
    'A00022727', 'A00011265', 'A00024546'
]

 # 82 controls that met requirement
met_requirement_control_ids = [
    'A00000300', 'A00002198', 'A00003150', 'A00004087', 'A00007409', 'A00010684', 'A00011265', 'A00011725',
    'A00012995', 'A00013140', 'A00013816', 'A00014839', 'A00014898', 'A00015759', 'A00015826', 'A00018553',
    'A00018716', 'A00019888', 'A00020805', 'A00020895', 'A00020968', 'A00020984', 'A00021058', 'A00021072',
    'A00021081', 'A00021085', 'A00022400', 'A00022490', 'A00022509', 'A00022592', 'A00022619', 'A00022653',
    'A00022687', 'A00022727', 'A00022729', 'A00022773', 'A00022810', 'A00022835', 'A00022837', 'A00022915',
    'A00023095', 'A00023120', 'A00023131', 'A00023143', 'A00023330', 'A00023337', 'A00023730', 'A00023800',
    'A00023848', 'A00023866', 'A00024160', 'A00024301', 'A00024372', 'A00024446', 'A00024535', 'A00024546', 
    'A00024663', 'A00024820', 'A00024932', 'A00024955', 'A00025969', 'A00026945', 'A00027487', 'A00027787', 
    'A00028052', 'A00028406', 'A00028409', 'A00029226', 'A00029452', 'A00031478', 'A00031764', 'A00033214', 
    'A00035751', 'A00036049', 'A00036555', 'A00036844', 'A00037007', 'A00037238', 'A00037318', 'A00037495', 
    'A00037564', 'A00037665'
]

In [ ]:
# Normalize and pad the data
def normalize_and_pad(data, max_t):
    # Normalize data to range [-1, 1]
    normalized = (data - np.min(data)) / (np.max(data) - np.min(data)) * 2 - 1
    # Pad the time dimension to max_t
    padded = np.pad(normalized, ((0, 0), (0, 0), (0, max_t - data.shape[2])), mode='constant')
    return padded

# Define a generator function to yield data batches
def data_generator(images, labels, batch_size):
    for i in range(0, len(images), batch_size):
        batch_images = images[i:i + batch_size]
        batch_labels = labels[i:i + batch_size]
        
        # Convert list of arrays to numpy array and add channel dimension
        # New shape should be (batch_size, time_steps, y, z, 1)
        batch_images = np.array(batch_images).transpose(0, 3, 1, 2)[..., np.newaxis]  # Adjust axes after x-axis collapse
        yield batch_images, np.array(batch_labels)

In [ ]:
train_loss_list = []
train_accuracy_list = []
test_loss_list = []
test_accuracy_list = []

# Run 15 iterations
for iteration in range(15):
    print(f"\n--- Iteration {iteration + 1} ---")

    # Training Data Selection
    train_ids_schiz = random.sample(met_requirement_schizophrenia_ids, 50)
    test_ids_schiz = [id for id in met_requirement_schizophrenia_ids if id not in train_ids_schiz]

    train_ids_control = random.sample(met_requirement_control_ids, 50)
    test_ids_control = [id for id in met_requirement_control_ids if id not in train_ids_control]
    test_ids_control = random.sample(test_ids_control, 9)

    # Classifier Test Data Selection
    classifier_test_ids = test_ids_schiz + test_ids_control

    # Specify the directory and file pattern
    directory_path = '../4D/'
    file_pattern = 'A*_????_func_FL_FD_RPI_DSP_MCF_SS_SM_Nui_CS_InStandard.nii.gz'

    # Construct the full path pattern
    path_pattern = f'{directory_path}/{file_pattern}'

    # Use glob to find all matching files
    matching_files = glob.glob(path_pattern)

    # Lists to store data for training and classification
    image_data_schiz = []
    image_data_control = []

    # Iterate through matching files and load data
    for file_path in matching_files:
        filename = os.path.basename(file_path)
        file_id = filename.split('_')[0]
        
        if file_id in train_ids_schiz or file_id in train_ids_control:
            t1_img = nib.load(file_path)
            t1_data = t1_img.get_fdata()
            
            # Filter out data with insufficient time points
            if t1_data.shape[3] < 90:
                continue

            # Collapse the x-axis by summing over it
            t1_data_collapsed = np.sum(t1_data, axis=0)  # Sum over x-axis

            # Store collapsed data in the appropriate list
            if file_id in train_ids_schiz:
                image_data_schiz.append(t1_data_collapsed)
            elif file_id in train_ids_control:
                image_data_control.append(t1_data_collapsed)

    # Display total loaded data counts
    print(f"Total control loaded: {len(image_data_control)}")
    print(f"Total schiz loaded: {len(image_data_schiz)}")

    # Determine the maximum time-dimension size
    max_t_size_schiz = max(img.shape[2] for img in image_data_schiz)  # Time dimension is now at index 2 after collapsing
    max_t_size_control = max(img.shape[2] for img in image_data_control)
    max_t_size = max(max_t_size_schiz, max_t_size_control)

    # Apply normalization and padding to all loaded data
    padded_data_schiz = [normalize_and_pad(img, max_t_size) for img in image_data_schiz]
    padded_data_control = [normalize_and_pad(img, max_t_size) for img in image_data_control]

    # Convert lists to numpy arrays for model input
    padded_data_array_schiz = np.array(padded_data_schiz)
    padded_data_array_control = np.array(padded_data_control)

    # Print the shape of the padded arrays
    print(f"Shape after normalization and padding (control): {padded_data_array_control[0].shape}")
    print(f"Shape after normalization and padding (schiz): {padded_data_array_schiz[0].shape}")

    # Batch size
    batch_size = 10

    # Create labels
    labels_schiz = np.ones(len(padded_data_array_schiz))
    labels_control = np.zeros(len(padded_data_array_control))

    # Combine images and labels
    train_images = np.concatenate((padded_data_array_schiz, padded_data_array_control), axis=0)  # Concatenate along the first dimension
    train_labels = np.concatenate((labels_schiz, labels_control), axis=0)

    # Shuffle indices
    indices = np.arange(len(train_images))
    np.random.shuffle(indices)

    # Shuffle data based on indices
    train_images = [train_images[i] for i in indices]
    train_labels = train_labels[indices]

    # Create TensorFlow Dataset from the generator
    train_dataset = tf.data.Dataset.from_generator(
        lambda: data_generator(train_images, train_labels, batch_size),
        output_signature=(
            tf.TensorSpec(shape=(None, max_t_size, 109, 91, 1), dtype=tf.float32),  # Adjusted shape for ConvLSTM3D after collapsing x-axis
            tf.TensorSpec(shape=(None,), dtype=tf.float32)
        )
    )

    # Prefetch for performance improvement
    train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    # Debug: Test the generator
    for images, labels in train_dataset.take(1):
        print(f"Batch image shape: {images.shape}")  # Should be (batch_size, max_t_size, 109, 91, 1)
        print(f"Batch labels shape: {labels.shape}")

    # Define the RNN model
    def build_rnn_model():
        # Define input shape: (time_steps, y, z, channels)
        time_steps = max_t_size  # Number of time points
        input_shape = (time_steps, 109, 91, 1)  # Shape after collapsing x-axis and adding channel

        # Input layer
        inputs = layers.Input(shape=input_shape)
        
        convlstm_out = layers.ConvLSTM2D(filters=16, kernel_size=(3, 3), padding='same', return_sequences=True, data_format='channels_last')(inputs)
        pooled_out = tf.keras.layers.TimeDistributed(tf.keras.layers.GlobalAveragePooling2D())(convlstm_out)
        dense_out = tf.keras.layers.Dense(256, activation='relu')(pooled_out)
        outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense_out)

        adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

        model = tf.keras.models.Model(inputs, outputs)
        model.compile(optimizer=adam_optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        
        return model

    # Create the model
    rnn_model = build_rnn_model()
    rnn_model.summary()

    # Metrics for training
    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')

    # Define number of epochs
    epochs = 75

    # Define a step function for training
    @tf.function
    def train_step(images, labels):
        with tf.GradientTape() as tape:
            predictions = rnn_model(images, training=True)
            # Compute binary cross-entropy loss
            loss = tf.keras.losses.binary_crossentropy(labels, predictions[:, -1, 0])

        # Apply gradients
        gradients = tape.gradient(loss, rnn_model.trainable_variables)
        rnn_model.optimizer.apply_gradients(zip(gradients, rnn_model.trainable_variables))

        # Update training metrics
        train_loss.update_state(loss)
        train_accuracy.update_state(labels, predictions[:, -1, 0])

    # Training loop
    for epoch in range(epochs):
        train_loss.reset_states()
        train_accuracy.reset_states()

        # Iterate through the training dataset
        for images, labels in train_dataset:
            train_step(images, labels)

        # Print the loss and accuracy for the current epoch
        print(f"Epoch {epoch + 1}, Loss: {train_loss.result().numpy()}, Training Accuracy: {train_accuracy.result().numpy()}")
    
    # Store training metrics for this iteration
    train_loss_list.append(train_loss.result().numpy())
    train_accuracy_list.append(train_accuracy.result().numpy())

    # List to store test images and labels
    test_image_data = []
    test_labels = []

    # Test IDs to filter relevant files
    test_ids = classifier_test_ids

    # Iterate through matching files and filter based on test IDs
    for file_path in matching_files:
        filename = os.path.basename(file_path)
        file_id = filename.split('_')[0]
        
        if file_id in test_ids:
            # Load the MRI image
            t1_img = nib.load(file_path)
            t1_data = t1_img.get_fdata()
            
            # Filter out images with insufficient time points
            if t1_data.shape[3] < 90:
                continue

            # Collapse the x-axis by summing over it
            t1_data_collapsed = np.sum(t1_data, axis=0)  # Sum over x-axis (axis=0)
            
            # Normalize the processed image (shape: (109, 91, t))
            processed_image_normalized = (t1_data_collapsed - np.min(t1_data_collapsed)) / (np.max(t1_data_collapsed) - np.min(t1_data_collapsed)) * 2 - 1

            # Pad or truncate the time dimension to match the expected size (max_t_size)
            current_t_size = processed_image_normalized.shape[2]
            if current_t_size < max_t_size:
                pad_size = max_t_size - current_t_size
                processed_image_padded = np.pad(
                    processed_image_normalized, 
                    ((0, 0), (0, 0), (0, pad_size)), 
                    mode='constant'
                )
            elif current_t_size > max_t_size:
                processed_image_padded = processed_image_normalized[:, :, :max_t_size]
            else:
                processed_image_padded = processed_image_normalized

            # Reshape to add channel dimension (shape: (109, 91, max_t_size, 1))
            processed_image_padded = np.expand_dims(processed_image_padded, axis=-1)  # Add channel dimension

            # Transpose to match (time, y, z, channels) for ConvLSTM2D (shape: (max_t_size, 109, 91, 1))
            processed_image_transposed = np.transpose(processed_image_padded, (2, 0, 1, 3))

            # Append to test image data
            test_image_data.append(processed_image_transposed)
            
            # Determine the label: 1 for schizophrenia, 0 for control
            label = 1 if file_id in met_requirement_schizophrenia_ids else 0
            test_labels.append(label)

    # Convert lists to numpy arrays for easier handling in TensorFlow
    test_images_array = np.array(test_image_data)
    test_labels_array = np.array(test_labels)

    # Create a TensorFlow dataset from the numpy arrays
    batch_size = 1
    test_dataset = tf.data.Dataset.from_tensor_slices((test_images_array, test_labels_array)).batch(batch_size)

    # Prefetch for performance improvement
    test_dataset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    # Debug: Check the shapes of the test data
    # for images, labels in test_dataset.take(1):
    #     print(f"Batch image shape: {images.shape}")  # Expected shape: (batch_size, max_t_size, 109, 91, 1)
    #     print(f"Batch labels shape: {labels.shape}")  # Expected shape: (batch_size,)

    def evaluation_step(images, labels):
        # Get predictions from the model
        predictions = rnn_model(images, training=False)
        
        # Compute binary cross-entropy loss
        loss = tf.keras.losses.binary_crossentropy(labels, predictions[:, -1, 0])
        
        # Update evaluation metrics
        test_loss.update_state(loss)
        test_accuracy.update_state(labels, predictions[:, -1, 0])

    # Initialize metrics for evaluation
    test_loss = tf.keras.metrics.Mean(name='test_loss')
    test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')

    # Reset states before evaluating
    test_loss.reset_states()
    test_accuracy.reset_states()

    # Evaluate on test_dataset without using strategy.run()
    for images, labels in test_dataset:
        evaluation_step(images, labels)

    # Store evaluation metrics for this iteration
    test_loss_list.append(test_loss.result().numpy())
    test_accuracy_list.append(test_accuracy.result().numpy())

    # Print evaluation results
    print(f"Test Loss: {test_loss.result().numpy()}, Test Accuracy: {test_accuracy.result().numpy()}")

    # Predict the probabilities on the test dataset
    predictions = rnn_model.predict(test_dataset)

    # Print shape of predictions for debugging
    print(f"Predictions shape: {predictions.shape}")

    # Convert probabilities to class labels (binary classification)
    predicted_labels = (predictions > 0.5).astype(int)

    # Extract the last time step prediction for each sequence
    predicted_labels_last = predicted_labels[:, -1, 0]  # Extract last time step and flatten

    # Print the shape of predicted_labels_last for debugging
    print(f"Predicted labels shape (last time step): {predicted_labels_last.shape}")

    actual_labels = test_labels_array
    # Extract actual labels for comparison (actual labels are already at the sequence level)
    actual_labels_per_sequence = actual_labels.flatten()  # Just flatten, no slicing needed

    # Print the shape of extracted actual labels for comparison
    print(f"Actual labels per sequence shape: {actual_labels_per_sequence.shape}")

    # Check if the lengths of actual and predicted labels match before evaluation
    if len(actual_labels_per_sequence) == len(predicted_labels_last):
        from sklearn.metrics import classification_report, confusion_matrix

        # Compute and print confusion matrix
        print("Confusion Matrix:")
        print(confusion_matrix(actual_labels_per_sequence, predicted_labels_last))
        
        # Compute and print classification report
        print("Classification Report:")
        print(classification_report(actual_labels_per_sequence, predicted_labels_last))
    else:
        print("The lengths of actual labels per sequence and predicted labels still don't match.")
        print(f"Length of actual labels per sequence: {len(actual_labels_per_sequence)}")
        print(f"Length of predicted labels: {len(predicted_labels_last)}")

    # Create a DataFrame to compare predicted vs. actual labels
    comparison_df = pd.DataFrame({
        'Predicted Labels': predicted_labels_last,
        'Actual Labels': actual_labels_per_sequence
    })

    # Print the full comparison DataFrame
    print(comparison_df)

average_train_loss = np.mean(train_loss_list)
average_train_accuracy = np.mean(train_accuracy_list)
average_test_loss = np.mean(test_loss_list)
average_test_accuracy = np.mean(test_accuracy_list)

# Print average results
print(f"\nAverage Training Loss over 15 iterations: {average_train_loss}")
print(f"Average Training Accuracy over 15 iterations: {average_train_accuracy}")
print(f"Average Test Loss over 15 iterations: {average_test_loss}")
print(f"Average Test Accuracy over 15 iterations: {average_test_accuracy}")
